# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [3]:
!pip install smdebug

  Using cached smdebug-1.0.12-py2.py3-none-any.whl (270 kB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp37-cp37m-manylinux2010_x86_64.whl (20 kB)
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.3
    Uninstalling urllib3-2.0.3:
      Successfully uninstalled urllib3-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.


In [4]:
!pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html --upgrade

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/rocm5.2/torch-1.13.1%2Brocm5.2-cp37-cp37m-linux_x86_64.whl (1342.9 MB)
  Using cached https://download.pytorch.org/whl/rocm5.2/torchvision-0.14.1%2Brocm5.2-cp37-cp37m-linux_x86_64.whl (70.8 MB)


In [5]:
!pip install jinja2==3.0.2

  Using cached Jinja2-3.0.2-py3-none-any.whl (133 kB)
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.2
    Uninstalling Jinja2-3.1.2:
      Successfully uninstalled Jinja2-3.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.
spyder 4.0.1 requires jedi==0.14.1, but you have jedi 0.18.2 which is incompatible.


In [6]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import os 
import gzip
import boto3
import random
import IPython
import numpy as np

import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile
from sagemaker.tuner import (IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner)

from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

from torchvision import transforms
from torchvision.datasets import CIFAR10

[2023-06-22 21:41:25.118 datascience-1-0-ml-m5-large-ab1c8cfd3029fb39f3ca96edc853:18 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


In [7]:
session = sagemaker.Session()

bucket= session.default_bucket()
print("Default Bucket: {}".format(bucket))

region = session.boto_region_name
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

Default Bucket: sagemaker-us-east-1-804746932029
AWS Region: us-east-1
RoleArn: arn:aws:iam::804746932029:role/service-role/AmazonSageMaker-ExecutionRole-20230607T163653


## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [ ]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!unzip dogImages.zip

In [ ]:
# Upload to S3

os.environ["DEFAULT_S3_BUCKET"] = bucket
!aws s3 sync ./dogImages s3://${DEFAULT_S3_BUCKET}/dogImages/

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [ ]:
inputs = session.upload_data(path="dogImages", bucket=bucket)
print("input spec (in this case, just an S3 path): {}".format(inputs))

In [76]:
inputs = "s3://sagemaker-us-east-1-804746932029/data"

In [ ]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([32, 64, 128, 256, 512]),
    "epochs": IntegerParameter(2, 4)
}

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Testing Loss: ([0-9\\.]+)"}]

In [ ]:
#TODO: Create estimators for your HPs
estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.large"
)

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=5,
    max_parallel_jobs=5,
    objective_type=objective_type,
)

In [ ]:
# TODO: Fit your HP Tuner
tuner.fit({"training": inputs})

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [74]:
rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
]

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)
debugger_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [75]:
hyperparameters= {
    "learning_rate": 0.0014377096270416446,
    "batch_size": 64,
    "epochs": 3
}


estimator = PyTorch(
    entry_point="train_model.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.large",
    hyperparameters=hyperparameters,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
    rules=rules
)

In [77]:
estimator.fit({"training": inputs}, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framewo

Using provided s3_resource


INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-06-22-01-30-16-167


2023-06-22 01:30:16 Starting - Starting the training job...
2023-06-22 01:30:37 Starting - Preparing the instances for trainingLossNotDecreasing: InProgress
VanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
......
2023-06-22 01:31:36 Downloading - Downloading input data.........
2023-06-22 01:33:17 Training - Downloading the training image...
2023-06-22 01:33:42 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-06-22 01:33:46,211 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-06-22 01:33:46,215 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-06-22 01:33:46,224 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2

In [79]:
training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

Training jobname: pytorch-training-2023-06-22-01-30-16-167
Region: us-east-1


In [90]:
tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-east-1-804746932029/', 'ProfilingIntervalInMilliseconds': 500, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'FileOpenFailThreshold': '50', 'HorovodProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'LocalPath': '/opt/ml/output/profiler', 'PythonProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760', 'SMDataParallelProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }'}, 'DisableProfiler': False}
s3 path:s3://sagemaker-us-east-1-804746932029/pytorch-training-2023-06-22-01-30-16-167/profiler-output


Profiler data from system is available
[2023-06-22 02:36:50.392 datascience-1-0-ml-m5-large-ab1c8cfd3029fb39f3ca96edc853:56 INFO metrics_reader_base.py:134] Getting 3

In [81]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

! aws s3 ls {rule_output_path} --recursive
! aws s3 cp {rule_output_path} ./ --recursive

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

You will find the profiler report in s3://sagemaker-us-east-1-804746932029/pytorch-training-2023-06-22-01-30-16-167/rule-output
2023-06-22 02:04:05     386688 pytorch-training-2023-06-22-01-30-16-167/rule-output/ProfilerReport/profiler-output/profiler-report.html
2023-06-22 02:04:05     236724 pytorch-training-2023-06-22-01-30-16-167/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2023-06-22 02:04:00        192 pytorch-training-2023-06-22-01-30-16-167/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2023-06-22 02:04:00        200 pytorch-training-2023-06-22-01-30-16-167/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2023-06-22 02:04:00       2009 pytorch-training-2023-06-22-01-30-16-167/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2023-06-22 02:04:00        127 pytorch-training-2023-06-22-01-30-16-167/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.jso

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [92]:
IPython.display.HTML(filename="ProfilerReport/profiler-output/profiler-report.html")

## Model Deploying

In [82]:
import sagemaker
from sagemaker.pytorch import PyTorchModel

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# loading estimator from s3
estimator = sagemaker.estimator.Estimator.attach('pytorch-training-2023-06-22-01-30-16-167')
model_location=estimator.model_data

# TODO: Deploy your model to an endpoint
model = PyTorchModel(
    entry_point="inference.py",
    role=role,
    model_data='s3://sagemaker-us-east-1-804746932029/pytorch-training-2023-06-22-01-30-16-167/output/model.tar.gz',
    framework_version="1.5.0",
    py_version="py3",
)

# SageMaker instance
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.c4.xlarge",
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-804746932029/pytorch-training-2023-06-22-01-30-16-167/output/model.tar.gz), script artifact (s3://sagemaker-us-east-1-804746932029/pytorch-training-2023-06-22-01-30-16-167/source/sourcedir.tar.gz), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-804746932029/pytorch-training-2023-06-22-02-09-36-362/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-training-2023-06-22-02-09-36-362
INFO:sagemaker:Creating endpoint-config with name pytorch-training-2023-06-22-02-09-36-362
INFO:sagemaker:Creating endpoint with name pytorch-training-2023-06-22-02-09-36-362


-------!

In [11]:
import torchvision
import torchvision.transforms as transforms
from PIL import Image

img_path = "dogImages/test/056.Dachshund/Dachshund_04003.jpg"

image = Image.open(img_path)

testing_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.RandomResizedCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

img_tensor = testing_transform(image)
img_tensor = img_tensor.unsqueeze(0)
img_tensor_pred = np.array(img_tensor)

In [90]:
# TODO: Run an prediction on the endpoint
response = predictor.predict({"inputs": img_tensor_pred})
response

[[-8.535566329956055,
  -7.350709438323975,
  -7.354972839355469,
  -7.112792491912842,
  -7.383466720581055,
  -9.17442512512207,
  -7.5022382736206055,
  -7.455461502075195,
  -7.247780799865723,
  -6.944657325744629,
  -6.325321674346924,
  -6.667513847351074,
  -8.637508392333984,
  -7.998115062713623,
  -6.714148044586182,
  -6.852627277374268,
  -7.854848384857178,
  -8.238541603088379,
  -8.985913276672363,
  -8.21902084350586,
  -9.491715431213379,
  -8.034934997558594,
  -7.9885029792785645,
  -9.789983749389648,
  -8.792864799499512,
  -7.6876091957092285,
  -7.78236198425293,
  -8.426137924194336,
  -8.716615676879883,
  -7.694022178649902,
  -7.8732099533081055,
  -8.3123779296875,
  -7.875696659088135,
  -8.25948715209961,
  -7.479219913482666,
  -6.6932501792907715,
  -8.309494018554688,
  -6.8721089363098145,
  -8.054428100585938,
  -6.906885623931885,
  -9.236309051513672,
  -7.835628032684326,
  -7.483212947845459,
  -7.1967854499816895,
  -6.7462239265441895,
  -6.613

[[-8.535566329956055,
  -7.350709438323975,
  -7.354972839355469,
  -7.112792491912842,
  -7.383466720581055,
  -9.17442512512207,
  -7.5022382736206055,
  -7.455461502075195,
  -7.247780799865723,
  -6.944657325744629,
  -6.325321674346924,
  -6.667513847351074,
  -8.637508392333984,
  -7.998115062713623,
  -6.714148044586182,
  -6.852627277374268,
  -7.854848384857178,
  -8.238541603088379,
  -8.985913276672363,
  -8.21902084350586,
  -9.491715431213379,
  -8.034934997558594,
  -7.9885029792785645,
  -9.789983749389648,
  -8.792864799499512,
  -7.6876091957092285,
  -7.78236198425293,
  -8.426137924194336,
  -8.716615676879883,
  -7.694022178649902,
  -7.8732099533081055,
  -8.3123779296875,
  -7.875696659088135,
  -8.25948715209961,
  -7.479219913482666,
  -6.6932501792907715,
  -8.309494018554688,
  -6.8721089363098145,
  -8.054428100585938,
  -6.906885623931885,
  -9.236309051513672,
  -7.835628032684326,
  -7.483212947845459,
  -7.1967854499816895,
  -6.7462239265441895,
  -6.613

In [93]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()